> Attention is all you need

ref: [딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/31379), [Attention is all you need](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)

In [ ]:
https://github.com/huggingface/transformers

In [ ]:
https://github.com/huggingface/transformers/blob/main/README_ko.md